In [1]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.


In [2]:
excel_path = "/Users/AlexanderLarsson/documents/VSCode/testdata_exjobb.xlsx"
indata_sheet = "indata"
indataDF = pd.read_excel(excel_path,indata_sheet)
outdataDF = indataDF.copy()
indataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point
0,1,RDC,Johannesburg,10,40,NaN,NaN,NaN,NaN,NaN,3
1,2,Dealer,Deal1,10,10,1.0,0.95,Normal,10.0,4.0,4
2,3,Dealer,Deal2,3,10,1.0,0.95,Normal,4.0,4.0,5
3,4,Dealer,Deal3,2,10,1.0,0.95,Poisson,4.0,4.0,6
4,5,Dealer,Deal4,4,10,1.0,0.95,NBD,3.0,6.0,7
5,6,Dealer,Deal5,5,10,1.0,0.95,NBD,3.0,4.0,8


In [3]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
Q_warehouse = int(indataDF.get(indataDF["Type"] == "RDC").get("Q").to_numpy())
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()
demand_mean_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
demand_stdev_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))
R_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Reorder point").to_numpy()
R_warehouse = int(indataDF.get(indataDF["Type"] == "RDC").get("Reorder point").to_numpy())


Calculating lead time demand mean and std dev for RDC

In [4]:
rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
demand_mean_warehouse = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = demand_mean_warehouse
demand_stdev_warehouse = math.sqrt(sigma2_L) * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = demand_stdev_warehouse

outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev
0,1,RDC,Johannesburg,10,40,NaN,NaN,Normal,24.0,3.103465,3,240.0,31.034653
1,2,Dealer,Deal1,10,10,1.0,0.95,Normal,10.0,4.000000,4,NaN,NaN
2,3,Dealer,Deal2,3,10,1.0,0.95,Normal,4.0,4.000000,5,NaN,NaN
3,4,Dealer,Deal3,2,10,1.0,0.95,Poisson,4.0,4.000000,6,NaN,NaN
4,5,Dealer,Deal4,4,10,1.0,0.95,NBD,3.0,6.000000,7,NaN,NaN
5,6,Dealer,Deal5,5,10,1.0,0.95,NBD,3.0,4.000000,8,NaN,NaN


Calculating lead times approximations for dealers

In [5]:
W = waiting_time(negative_inventory(Q_subbatch_size,Q_warehouse,R_warehouse,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
L_dealer_arr = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = L_dealer_arr
Ldemand_dealer_mean_arr = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = Ldemand_dealer_mean_arr
Ldemand_dealer_stdev_arr = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] = Ldemand_dealer_stdev_arr
outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev,Waiting time,Lead time
0,1,RDC,Johannesburg,10,40,NaN,NaN,Normal,24.0,3.103465,3,240.000000,31.034653,NaN,NaN
1,2,Dealer,Deal1,10,10,1.0,0.95,Normal,10.0,4.000000,4,119.400861,47.760344,1.940086,11.940086
2,3,Dealer,Deal2,3,10,1.0,0.95,Normal,4.0,4.000000,5,19.760344,19.760344,1.940086,4.940086
3,4,Dealer,Deal3,2,10,1.0,0.95,Poisson,4.0,4.000000,6,15.760344,15.760344,1.940086,3.940086
4,5,Dealer,Deal4,4,10,1.0,0.95,NBD,3.0,6.000000,7,17.820258,35.640516,1.940086,5.940086
5,6,Dealer,Deal5,5,10,1.0,0.95,NBD,3.0,4.000000,8,20.820258,27.760344,1.940086,6.940086


Calculating fill rate for RDC

In [6]:
#Need fill rate calculations for gamma in service_level_computations


if wh_dist == "Normal":
    outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = fill_rate_normal_demand(R_warehouse, Q_warehouse, demand_mean_warehouse, demand_stdev_warehouse)
#elif wh_dist == "Gamma":
    #outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = #add here
#elif wh_dist == "NBD":
    #outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = fill_rate_compound_poisson_demand(demand_prob_arr_negative_binomial(L_wh, demand_mean_warehouse, demand_stdev_warehouse), IL_prob_array_discrete_positive(R_warehouse, Q_warehouse, demand_prob_arr_negative_binomial(L_wh, demand_mean_warehouse, demand_stdev_warehouse)))

outdataDF


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev,Waiting time,Lead time,Fill rate
0,1,RDC,Johannesburg,10,40,NaN,NaN,Normal,24.0,3.103465,3,240.000000,31.034653,NaN,NaN,0.475
1,2,Dealer,Deal1,10,10,1.0,0.95,Normal,10.0,4.000000,4,119.400861,47.760344,1.940086,11.940086,NaN
2,3,Dealer,Deal2,3,10,1.0,0.95,Normal,4.0,4.000000,5,19.760344,19.760344,1.940086,4.940086,NaN
3,4,Dealer,Deal3,2,10,1.0,0.95,Poisson,4.0,4.000000,6,15.760344,15.760344,1.940086,3.940086,NaN
4,5,Dealer,Deal4,4,10,1.0,0.95,NBD,3.0,6.000000,7,17.820258,35.640516,1.940086,5.940086,NaN
5,6,Dealer,Deal5,5,10,1.0,0.95,NBD,3.0,4.000000,8,20.820258,27.760344,1.940086,6.940086,NaN


Calculating fill rates for dealers

In [8]:

# Normal demand works, problems with NBD and Poisson
fill_rate_dealer_arr = []

for i in range(0, len(demand_type_arr)):
    if demand_type_arr[i] == "Normal":
        fill_rate_dealer_arr.append(fill_rate_normal_demand(R_dealer_arr[i], Q_dealer_arr[i], Ldemand_dealer_mean_arr[i], Ldemand_dealer_stdev_arr[i]))
    elif demand_type_arr[i] == "Poisson":
        fill_rate_dealer_arr.append(fill_rate_poisson_demand(IL_prob_array_discrete_positive(R_dealer_arr[i], Q_dealer_arr[i], demand_prob_arr_poisson(L_dealer_arr[i], Ldemand_dealer_mean_arr[i]))))
    elif demand_type_arr[i] == "NBD":
         fill_rate_dealer_arr.append(fill_rate_compound_poisson_demand(demand_prob_arr_negative_binomial(L_dealer_arr, Ldemand_dealer_mean_arr, Ldemand_dealer_stdev_arr**2), IL_prob_array_discrete_positive(R_dealer_arr[i], Q_dealer_arr[i], demand_prob_arr_negative_binomial(L_dealer_arr, Ldemand_dealer_mean_arr, Ldemand_dealer_stdev_arr**2))))

outdataDF.loc[outdataDF["Type"]== "Dealer", "Fill rate"] = fill_rate_dealer_arr
outdataDF

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()